In [47]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random, randint,sample,uniform
from copy import deepcopy
import math
from scipy import spatial
import time
%matplotlib inline

def initialize_data(filename):
    #readData
    train_data = pd.read_csv(filename)
    
    #normalize the datasets
    train_nor = (train_data -train_data.mean())/train_data.std()
    
    #convert dataFrame into dataList and show
    train_nor["Group"] = 0
    dataList = train_nor.values.tolist()
    return dataList
    
def generate_centroid(k,dataList):
        clusters = list()
        First = deepcopy(dataList[randint(0,len(dataList)-1)])[0:-1]
        clusters.append(First)
        while(len(clusters) != k):
            sum_distance = 0
            disList = list()
            for j in dataList:
                mins = 100000
                dis = 0
                for i in range(len(clusters)):
                    dis = calculate(j,clusters[i])
                    if dis < mins:
                        mins = dis
                sum_distance = sum_distance + mins**(2)
                disList.append(mins**(2))
            tem = uniform(0,sum_distance) 
            time = 0 
            while(tem>0):
                tem = tem - disList[time]
                time = time + 1
            clusters.append(deepcopy(dataList[time-1])[0:-1])
        return clusters

def kmeans_generate_centroid(k,dataList):
    clusters = list()
    for i in range(k):
        First = deepcopy(dataList[randint(0,len(dataList)-1)])[0:-1]
        clusters.append(First)
    return clusters
    
def calculate(record_1,record_2):
    distance = 0
    for x in range(10):
        distance += (record_1[x]-record_2[x])**(2)
    return distance**(0.5)

def update_group(dataList,clusters):
    while True:
        changed = 0;
        for p in dataList:
            mins = 10000000
            ori_group = p[-1]
            for i in range(len(clusters)):
                dis = calculate(p,clusters[i])
                if (dis < mins):
                   mins = dis
                   p[-1] = i+1
            if(ori_group != p[-1]):
                changed = changed + 1
        update_centroid(dataList,clusters)
        if(changed == 0):
                break;
def update_centroid(dataList,clusters):
    for i in range(len(clusters)):
        sum_list = [0 for i in range(len(clusters[i]))]
        num = 0
        for j in dataList:
            if(j[-1] == (i+1)):
                num = num + 1
                for k in range(len(j)-1):
                    sum_list[k] = sum_list[k] + j[k]
        #print("cluster:",(i+1),num)
        for n in range(len(clusters[0])):
            if num > 0:
                clusters[i][n] = sum_list[n]/num
    
def calculate_sumDistance(dataList,clusters):
    sum_dis = 0
    for i in dataList:
        group = i[-1]-1
        dis = calculate(i,clusters[group])
        sum_dis += dis
        
    return sum_dis
   
def kmeansPlus(filename,k):
    dataList = initialize_data(filename)
    clusters = generate_centroid(k,dataList)
    update_group(dataList,clusters)
    dis = calculate_sumDistance(dataList,clusters)
    DVI = DuneIndex(dataList,clusters)
    DB = BouldinIndex(dataList,clusters)
    df = pd.DataFrame(dataList)
    print("sum_dis:",dis)
    print("DVI:",DVI)
    print("DB:",DB)
    return dis

def kmeans(filename,k):
    dataList = initialize_data(filename)
    clusters = kmeans_generate_centroid(k,dataList)
    update_group(dataList,clusters)
    dis = calculate_sumDistance(dataList,clusters)
    DVI = DuneIndex(dataList,clusters)
    DB = BouldinIndex(dataList,clusters)
    df = pd.DataFrame(dataList)
    print("sum_dis:",dis)
    print("DVI:",DVI)
    print("DB:",DB)
    return dis

#measures to evaluate
#DuneIndex
def DuneIndex(dataList,clusters):
    minintro_dis = 10000000
    maxinner_dis = 0
    for j in clusters:
        for k in clusters:
            if(j != k):
                dis = calculate(j,k)
                if (dis <minintro_dis):
                    minintro_dis = dis
    for m in dataList:
        for n in dataList:
            if (m[-1] == n[-1]):
                dis = calculate(m,n)
                if (dis > maxinner_dis):
                    maxinner_dis = dis
    DVI = minintro_dis**(0.5) / maxinner_dis**(0.5)
    return DVI

def calculate_averageDis(dataList,cluster,index):
    sum_dis = 0
    count = 0
    for i in dataList:
        if(i[-1] == index):
            dis = calculate(i,cluster)
            sum_dis = sum_dis + dis
            count = count + 1
    ave = sum_dis/count
    return ave

#Davis Index
def BouldinIndex(dataList,clusters):
    sum_dis = 0
    for i in range(len(clusters)):
        max_dis = 0
        for j in range(len(clusters)):
            if (i != j):
                dis_1 = calculate_averageDis(dataList,clusters[i],i+1) + calculate_averageDis(dataList,clusters[j],j+1)
                dis_2 = calculate(clusters[i],clusters[j])
                result = dis_1 / dis_2
                if (result > max_dis):
                    max_dis = result
        sum_dis = sum_dis + max_dis
    DB = sum_dis / len(clusters)
    return DB
    
final = 0
min_dis = 10000
total_time = 0
for i in range(10):
    start_time = time.time()
    #dis = kmeansPlus("cloudData.csv",50)
    dis2 = kmeans("cloudData.csv",50)
    run_time = time.time()-start_time
    print("running time:",run_time)
    final += dis2
    total_time = total_time + run_time
average = final / 10
print("everage distance:",average)
print("everage time:",total_time/10)

sum_dis: 1066.4858585571126
DVI: 0.44443934881793357
DB: 1.0420064256703199
running time: 20.169259071350098
sum_dis: 1067.1392527118603
DVI: 0.40708356490627096
DB: 1.052046273457634
running time: 16.135871410369873
sum_dis: 1067.0845443202388
DVI: 0.3836017397542227
DB: 1.0712905651617621
running time: 15.412123680114746
sum_dis: 1062.3703591825376
DVI: 0.424124011553921
DB: 1.0606858556761067
running time: 15.817102909088135
sum_dis: 1067.841889917625
DVI: 0.41950322697582293
DB: 1.0896950562791208
running time: 21.38218379020691
sum_dis: 1071.1660745472461
DVI: 0.44809042839807267
DB: 1.0507299038678266
running time: 13.212323427200317
sum_dis: 1075.1520219701788
DVI: 0.42730541290138924
DB: 1.0787851084859397
running time: 18.113230228424072
sum_dis: 1050.715437296431
DVI: 0.41797192070033473
DB: 1.0363192618892918
running time: 14.941414594650269
sum_dis: 1056.6064238121076
DVI: 0.3860487219032932
DB: 1.0477922098566492
running time: 15.020453929901123
sum_dis: 1108.524939189231


In [33]:
sum_list = [0 for i in range(10)]
print(sum_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
